In [1]:
import os
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.splits import create_splits_scenes
from PIL import Image
import torch
from ultralytics import YOLO
import numpy as np
import json
from tqdm import tqdm


class NuScenesConesDataset:
    def __init__(self, dataroot, version="v1.0-mini", split="train"):
        self.nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
        self.split = split
        self.scenes = create_splits_scenes()[split]

    def prepare_dataset(self, output_dir, max_images=None):
        """
        Prepare dataset in YOLO format:
        - images/train/
        - images/val/
        - labels/train/
        - labels/val/

        Args:
            output_dir (str): Directory to save the processed dataset
            max_images (int, optional): Maximum number of images to process. If None, process all images.
        """
        os.makedirs(os.path.join(output_dir, "images", self.split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", self.split), exist_ok=True)

        image_count = 0

        # Track all annotations
        for scene in tqdm(self.scenes):
            # start with first scene
            scene_rec = self.nusc.scene[0]
            first_sample_token = scene_rec["first_sample_token"]
            sample = self.nusc.get("sample", first_sample_token)

            while sample:
                # Check if we've reached the maximum number of images
                if max_images is not None and image_count >= max_images:
                    print(f"Reached maximum number of images ({max_images})")
                    return

                # Get camera images
                cam_front_data = self.nusc.get(
                    "sample_data", sample["data"]["CAM_FRONT"]
                )

                # Get image
                img_path = os.path.join(self.nusc.dataroot, cam_front_data["filename"])
                img = Image.open(img_path)

                # Get annotations
                annotations = []
                for ann_token in sample["anns"]:
                    ann_rec = self.nusc.get("sample_annotation", ann_token)
                    if ann_rec["category_name"] == "movable_object.traffic_cone":
                        # Get 2D bbox in image coordinates
                        bbox = self.nusc.get_box(ann_rec["token"])
                        corners_2d = self.nusc.box_to_keypoints(
                            cam_front_data["token"], bbox
                        )

                        # Convert to YOLO format (normalized coordinates)
                        x_min, y_min = np.min(corners_2d, axis=0)
                        x_max, y_max = np.max(corners_2d, axis=0)

                        width = img.width
                        height = img.height

                        # YOLO format: <class> <x_center> <y_center> <width> <height>
                        x_center = ((x_min + x_max) / 2) / width
                        y_center = ((y_min + y_max) / 2) / height
                        bbox_width = (x_max - x_min) / width
                        bbox_height = (y_max - y_min) / height

                        annotations.append(
                            f"0 {x_center} {y_center} {bbox_width} {bbox_height}"
                        )

                # Save image and labels
                img_filename = os.path.basename(cam_front_data["filename"])
                label_filename = os.path.splitext(img_filename)[0] + ".txt"

                img.save(os.path.join(output_dir, "images", self.split, img_filename))
                with open(
                    os.path.join(output_dir, "labels", self.split, label_filename), "w"
                ) as f:
                    f.write("\n".join(annotations))

                image_count += 1

                # Move to next sample
                if sample["next"] == "":
                    break
                sample = self.nusc.get("sample", sample["next"])

In [2]:
# Set paths
dataroot = "../data/v1.0-mini"  # Replace with your NuScenes data path
output_dir = "../output_5"  # Replace with your desired output path

# Prepare training data
print("Preparing training data...")
train_dataset = NuScenesConesDataset(dataroot, split="train")
train_dataset.prepare_dataset(output_dir, max_images=10)

# Prepare validation data
print("Preparing validation data...")
val_dataset = NuScenesConesDataset(dataroot, split="val")
val_dataset.prepare_dataset(output_dir, max_images=10)

Preparing training data...
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.404 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


  0%|          | 0/700 [00:00<?, ?it/s]

Reached maximum number of images (10)
Preparing validation data...
Loading NuScenes tables for version v1.0-mini...


23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.464 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


  0%|          | 0/150 [00:00<?, ?it/s]

Reached maximum number of images (10)


In [5]:
import os
from ultralytics import YOLO
from PIL import Image
import torch
import yaml
from tqdm import tqdm


class YOLOConeDetector:
    def __init__(self, data_yaml_path=None):
        """
        Initialize the YOLO cone detector
        Args:
            data_yaml_path (str): Path to data.yaml file. If None, will create one.
        """
        self.model = None
        self.data_yaml_path = data_yaml_path

    def create_data_yaml(self, output_dir):
        """
        Create data.yaml file for YOLO training
        """
        # Convert to absolute path
        output_dir = os.path.abspath(output_dir)

        # Verify directories exist
        train_path = os.path.join(output_dir, "images", "train")
        val_path = os.path.join(output_dir, "images", "val")

        if not os.path.exists(train_path):
            raise FileNotFoundError(
                f"Training images directory not found at: {train_path}"
            )
        if not os.path.exists(val_path):
            raise FileNotFoundError(
                f"Validation images directory not found at: {val_path}"
            )

        data_yaml = {
            "train": os.path.join(output_dir, "images", "train"),
            "val": os.path.join(output_dir, "images", "val"),
            "test": os.path.join(output_dir, "images", "val"),  # Using val as test set
            "names": {0: "traffic_cone"},
            "nc": 1,  # number of classes
        }

        yaml_path = os.path.join(output_dir, "data.yaml")
        with open(yaml_path, "w") as f:
            yaml.dump(data_yaml, f)

        print(f"Created data.yaml at: {yaml_path}")
        print("Dataset paths:")
        print(f"Train: {data_yaml['train']}")
        print(f"Val: {data_yaml['val']}")

        self.data_yaml_path = yaml_path
        return yaml_path

    def verify_dataset(self, output_dir):
        """
        Verify that the dataset directory structure is correct
        """
        required_dirs = [
            os.path.join(output_dir, "images", "train"),
            os.path.join(output_dir, "images", "val"),
            os.path.join(output_dir, "labels", "train"),
            os.path.join(output_dir, "labels", "val"),
        ]

        for dir_path in required_dirs:
            if not os.path.exists(dir_path):
                raise FileNotFoundError(f"Required directory not found: {dir_path}")

            # Check if directory contains files
            if len(os.listdir(dir_path)) == 0:
                raise ValueError(f"Directory is empty: {dir_path}")

        print(
            "Dataset verification complete - all required directories exist and contain files."
        )

    def train(self, output_dir, epochs=100, batch_size=16, imgsz=640):
        """
        Train YOLO model on the dataset
        Args:
            output_dir (str): Directory containing the processed dataset
            epochs (int): Number of training epochs
            batch_size (int): Batch size for training
            imgsz (int): Image size for training
        """
        # Convert to absolute path
        output_dir = os.path.abspath(output_dir)

        # Verify dataset structure
        print("Verifying dataset structure...")
        self.verify_dataset(output_dir)

        if self.data_yaml_path is None:
            print("Creating data.yaml file...")
            self.data_yaml_path = self.create_data_yaml(output_dir)

        # Initialize YOLO model
        print("Initializing YOLO model...")
        self.model = YOLO("yolov8n.pt")  # Start with pre-trained YOLOv8n model

        # Train the model
        print(f"Starting training with data config: {self.data_yaml_path}")
        self.model.train(
            data=self.data_yaml_path,
            epochs=epochs,
            batch=batch_size,
            imgsz=imgsz,
            device="cuda" if torch.cuda.is_available() else "cpu",
            patience=50,  # Early stopping patience
            save=True,  # Save best model
            project=os.path.join(output_dir, "runs"),
            name="cone_detection",
        )

    def predict(self, image_path, conf_threshold=0.25):
        """
        Perform inference on a single image
        Args:
            image_path (str): Path to the image
            conf_threshold (float): Confidence threshold for detection
        Returns:
            list: List of detections (boxes, scores, classes)
        """
        if self.model is None:
            raise ValueError(
                "Model not trained or loaded. Please train or load a model first."
            )

        # Convert to absolute path
        image_path = os.path.abspath(image_path)
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at: {image_path}")

        results = self.model.predict(source=image_path, conf=conf_threshold, save=False)

        return results[0]  # Return first (and only) image results

    def evaluate(self, val_dataset_path):
        """
        Evaluate the model on validation dataset
        Args:
            val_dataset_path (str): Path to validation dataset
        """
        if self.model is None:
            raise ValueError(
                "Model not trained or loaded. Please train or load a model first."
            )

        metrics = self.model.val(data=self.data_yaml_path)
        return metrics

    def load_model(self, weights_path):
        """
        Load a trained YOLO model
        Args:
            weights_path (str): Path to the weights file
        """
        weights_path = os.path.abspath(weights_path)
        if not os.path.exists(weights_path):
            raise FileNotFoundError(f"Weights file not found at: {weights_path}")

        self.model = YOLO(weights_path)

In [6]:
detector = YOLOConeDetector()
detector.train(output_dir=output_dir, epochs=50, batch_size=8)

Verifying dataset structure...
Dataset verification complete - all required directories exist and contain files.
Creating data.yaml file...
Created data.yaml at: /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/data.yaml
Dataset paths:
Train: /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/images/train
Val: /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/images/val
Initializing YOLO model...
Starting training with data config: /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/data.yaml
New https://pypi.org/project/ultralytics/8.3.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.20 🚀 Python-3.11.5 torch-2.4.1 CPU (Apple M2 Max)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/data.yaml, epochs=50, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=/Users/jiayizhou/Desktop/code_c

Fontconfig warning: ignoring UTF-8: not a valid region tag


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/labels/train... 10 images, 10 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 2242.10it/s]

train: New cache created: /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/labels/train.cache
WARNING ⚠️ No labels found in /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

                   all         10          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



50 epochs completed in 0.034 hours.
Optimizer stripped from /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/runs/cone_detection/weights/last.pt, 6.2MB
Optimizer stripped from /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/runs/cone_detection/weights/best.pt, 6.2MB

Validating /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/runs/cone_detection/weights/best.pt...
Ultralytics 8.3.20 🚀 Python-3.11.5 torch-2.4.1 CPU (Apple M2 Max)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]

                   all         10          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.3ms preprocess, 25.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /Users/jiayizhou/Desktop/code_capstone/YOLONuScenes/output_5/runs/cone_detection
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
